# Problem statement
## Context
With the introduction of 3D animation, 2D animation started to be considered as obsolete in the western market. However, japanese animation (anime) is still prevalent with 2D hand-drawn animation, which is still evolving with introduciton of new genres and recent animation studios appearing there in Japan. To legally broadcast anime outside of Japan the licensors have to buy rights for the specific anime. Since their business is strongly dependant on the popularity of the anime they bought rights on, it is useful to predict the popularity of it.

## Goal
Identify factors that might help predict popularity of the anime (without considering synopsis).

# Data Collection
## Data Source
I used "Anime data" dataset from Kaggle provided by user [Canggih P Wibowo](https://www.kaggle.com/canggih/anime-data-score-staff-synopsis-and-genre).

## Data Description


In [1]:
import numpy as np
import pandas as pd 

import re




In [2]:
data = pd.read_csv('dataanime.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1563 entries, 0 to 1562
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Title            1563 non-null   object 
 1   Type             1563 non-null   object 
 2   Episodes         1563 non-null   object 
 3   Status           1563 non-null   object 
 4   Start airing     1563 non-null   object 
 5   End airing       1563 non-null   object 
 6   Starting season  1563 non-null   object 
 7   Broadcast time   1563 non-null   object 
 8   Producers        1563 non-null   object 
 9   Licensors        1563 non-null   object 
 10  Studios          1563 non-null   object 
 11  Sources          1563 non-null   object 
 12  Genres           1563 non-null   object 
 13  Duration         1563 non-null   object 
 14  Rating           1563 non-null   object 
 15  Score            1563 non-null   float64
 16  Scored by        1563 non-null   int64  
 17  Members       

In [3]:
data.head()

,Title,Type,Episodes,Status,Start airing,End airing,Starting season,Broadcast time,Producers,Licensors,Studios,Sources,Genres,Duration,Rating,Score,Scored by,Members,Favorites,Description
0,Fullmetal Alchemist: Brotherhood,TV,64,Finished Airing,2009-4-5,2010-7-4,Spring,Sundays at 17:00 (JST),"Aniplex,Square Enix,Mainichi Broadcasting Syst...","Funimation,Aniplex of America",Bones,Manga,"Action,Military,Adventure,Comedy,Drama,Magic,F...",24 min. per ep.,R,9.25,719706,1176368,105387,"""In order for something to be obtained, someth..."
1,Kimi no Na wa.,Movie,1,Finished Airing,2016-8-26,-,-,-,"Kadokawa Shoten,Toho,Sound Team Don Juan,Lawso...","Funimation,NYAV Post",CoMix Wave Films,Original,"Supernatural,Drama,Romance,School",1 hr. 46 min.,PG-13,9.19,454969,705186,33936,"Mitsuha Miyamizu, a high school girl, yearns t..."
2,Gintama°,TV,51,Finished Airing,2015-4-8,2016-3-30,Spring,Wednesdays at 18:00 (JST),"TV Tokyo,Aniplex,Dentsu","Funimation,Crunchyroll",Bandai Namco Pictures,Manga,"Action,Comedy,Historical,Parody,Samurai,Sci-Fi...",24 min. per ep.,R,9.16,70279,194359,5597,"Gintoki, Shinpachi, and Kagura return as the f..."
3,Steins;Gate 0,TV,23,Currently Airing,2018-4-12,-,Spring,Thursdays at 01:35 (JST),Nitroplus,Funimation,White Fox,Visual novel,"Sci-Fi,Thriller",23 min. per ep.,PG-13,9.16,12609,186331,1117,The dark untold story of Steins;Gate that lead...
4,Steins;Gate,TV,24,Finished Airing,2011-4-6,2011-9-14,Spring,Wednesdays at 02:05 (JST),"Frontier Works,Media Factory,Movic,AT-X,Kadoka...",Funimation,White Fox,Visual novel,"Sci-Fi,Thriller",24 min. per ep.,PG-13,9.14,552791,990419,90365,The self-proclaimed mad scientist Rintarou Oka...


**We can see that 'Episodes' are object values, however, it looks like they have to be int64. Let's check this column.**

In [4]:
data.Episodes.unique()

array(['64', '1', '51', '23', '24', '110', '148', '22', '13', '7', '12',
       '201', '10', '25', '26', '75', '4', '11', '43', '27', '37', '39',
       '47', '99', '120', '101', '-', '2', '50', '112', '62', '33', '94',
       '3', '8', '14', '40', '15', '77', '291', '203', '79', '6', '70',
       '38', '500', '103', '153', '102', '45', '21', '63', '175', '52',
       '5', '28', '145', '36', '78', '60', '69', '178', '35', '34', '109',
       '113', '114', '9', '49', '61', '97', '366', '167', '54', '358',
       '104', '20', '220', '48', '42', '46', '100', '142', '373', '131',
       '195', '1787', '19', '17', '147', '73', '127', '74', '16', '150',
       '140', '76', '53', '98', '29', '115', '18', '124', '224', '44',
       '154', '58'], dtype=object)

**Now, we know it's because there were some empty entries where value is '-'. Let's look at them**

In [5]:
data.loc[(data.Episodes == '-')]

,Title,Type,Episodes,Status,Start airing,End airing,Starting season,Broadcast time,Producers,Licensors,Studios,Sources,Genres,Duration,Rating,Score,Scored by,Members,Favorites,Description
84,One Piece,TV,-,Currently Airing,1999-10-20,-,Fall,Sundays at 09:30 (JST),"Fuji TV,TAP,Shueisha","Funimation,4Kids Entertainment",Toei Animation,Manga,"Action,Adventure,Comedy,Super Power,Drama,Fant...",24 min.,PG-13,8.54,417652,707815,68985,"Gol D. Roger was known as the ""Pirate King,"" t..."
248,Detective Conan,TV,-,Currently Airing,1996-1-8,-,Winter,Saturdays at 18:00 (JST),"Yomiuri Telecasting,Animax,YTV,TMS-Kyokuchi,Sh...","Funimation,Crunchyroll",TMS Entertainment,Manga,"Adventure,Mystery,Comedy,Police,Shounen",25 min.,PG-13,8.26,76893,154190,7626,"Shinichi Kudou, a great mystery expert at only..."
399,Uchuu Senkan Yamato 2202: Ai no Senshi-tachi,OVA,-,Currently Airing,2017-2-25,-,-,-,-,Funimation,Xebec,Original,"Action,Military,Sci-Fi,Space,Drama",25 min.,PG-13,8.11,732,8003,21,Space Battleship Yamato 2202: Warriors of Lov...
781,Captain Tsubasa (2018),TV,-,Currently Airing,2018-4-3,-,Spring,Tuesdays at 01:35 (JST),-,Viz Media,David Production,Manga,"Action,Sports,Shounen",22 min.,G,7.82,3214,23997,56,Captain Tsubasa is the passionate story of an ...
1202,Major 2nd (TV),TV,-,Currently Airing,2018-4-7,-,Spring,Saturdays at 17:35 (JST),"NHK,Shogakukan",-,NHK Enterprises,Manga,"Comedy,Sports,Drama,Shounen",24 min.,PG-13,7.62,757,7585,28,"Daigo is born as the son of Gorou, a father wh..."
1299,Mi Yu Xing Zhe,ONA,-,Currently Airing,2018-2-3,-,-,-,Jay Zone Comic,-,Thundray,Web manga,"Mystery,Psychological,Thriller",23 min.,R,7.58,284,2295,7,"A magician, a female doctor, a gangster, an ar..."
1315,Emiya-san Chi no Kyou no Gohan,ONA,-,Currently Airing,2017-12-31,-,-,-,"Aniplex,Kadokawa",Aniplex of America,ufotable,Manga,"Slice of Life,Comedy",13 min.,PG-13,7.57,5530,23605,57,"""Fulfilling meal...a familiar view.” \n\nIn th..."
1346,Doraemon (2005),TV,-,Currently Airing,2005-4-22,-,Spring,Fridays at 19:00 (JST),TV Asahi,-,"Studio Pierrot,Shin-Ei Animation",Manga,"Sci-Fi,Comedy,Kids,Shounen",25 min.,PG,7.56,1930,4756,64,Doraemon (2005) is the most recent anime serie...
1384,Inazuma Eleven: Ares no Tenbin,TV,-,Currently Airing,2018-4-6,-,Spring,Fridays at 17:55 (JST),-,-,OLM,Game,Sports,23 min.,G,7.56,593,8261,47,Inazuma Eleven Ares will be set in a parallel ...
1429,Hisone to Maso-tan,TV,-,Currently Airing,2018-4-13,-,Spring,Fridays at 00:00 (JST),-,-,Bones,Original,"Military,Fantasy",24 min.,PG-13,7.54,691,12482,8,Straightforward and innocent Hisone Amakasu is...


## Data cleaning 
Before starting Exploratory Data Analysis, I have to perform following actions:
1.  Correctly parse dates
2.  Change all '-' values to Nan
3.  Drop currently airing because they don't show final state of the score
4.  Change dtype of 'Episodes' to int64
5.  One-hot encode 'Genres', 'Producers', 'Licensors' and 'Studios'.
6.  Convert hours to minutes in 'duration' and change its dtype to int64

In [6]:
#1.  Correctly parse dates
def dateparse(x):
    try:
        a = pd.to_datetime(x, format='%Y-%m-%d')
        return a
    except:
        pass
        return np.nan

data = pd.read_csv('dataanime.csv', parse_dates={'Start air': ['Start airing'], 'End air': ['End airing']}, date_parser=dateparse)

data.head()

,Start air,End air,Title,Type,Episodes,Status,Starting season,Broadcast time,Producers,Licensors,Studios,Sources,Genres,Duration,Rating,Score,Scored by,Members,Favorites,Description
0,2009-04-05,2010-07-04,Fullmetal Alchemist: Brotherhood,TV,64,Finished Airing,Spring,Sundays at 17:00 (JST),"Aniplex,Square Enix,Mainichi Broadcasting Syst...","Funimation,Aniplex of America",Bones,Manga,"Action,Military,Adventure,Comedy,Drama,Magic,F...",24 min. per ep.,R,9.25,719706,1176368,105387,"""In order for something to be obtained, someth..."
1,2016-08-26,NaT,Kimi no Na wa.,Movie,1,Finished Airing,-,-,"Kadokawa Shoten,Toho,Sound Team Don Juan,Lawso...","Funimation,NYAV Post",CoMix Wave Films,Original,"Supernatural,Drama,Romance,School",1 hr. 46 min.,PG-13,9.19,454969,705186,33936,"Mitsuha Miyamizu, a high school girl, yearns t..."
2,2015-04-08,2016-03-30,Gintama°,TV,51,Finished Airing,Spring,Wednesdays at 18:00 (JST),"TV Tokyo,Aniplex,Dentsu","Funimation,Crunchyroll",Bandai Namco Pictures,Manga,"Action,Comedy,Historical,Parody,Samurai,Sci-Fi...",24 min. per ep.,R,9.16,70279,194359,5597,"Gintoki, Shinpachi, and Kagura return as the f..."
3,2018-04-12,NaT,Steins;Gate 0,TV,23,Currently Airing,Spring,Thursdays at 01:35 (JST),Nitroplus,Funimation,White Fox,Visual novel,"Sci-Fi,Thriller",23 min. per ep.,PG-13,9.16,12609,186331,1117,The dark untold story of Steins;Gate that lead...
4,2011-04-06,2011-09-14,Steins;Gate,TV,24,Finished Airing,Spring,Wednesdays at 02:05 (JST),"Frontier Works,Media Factory,Movic,AT-X,Kadoka...",Funimation,White Fox,Visual novel,"Sci-Fi,Thriller",24 min. per ep.,PG-13,9.14,552791,990419,90365,The self-proclaimed mad scientist Rintarou Oka...


In [7]:
#2.  Change all '-' values to Nan
data = data.replace('-', np.nan)
print('Fraction of missinng values :\n', data.isna().sum() / len(data.index) * 100)

Fraction of missinng values :
 Start air           0.895713
End air            37.428023
Title               0.000000
Type                0.000000
Episodes            0.703775
Status              0.000000
Starting season    47.472809
Broadcast time     64.555342
Producers          24.056302
Licensors          38.835573
Studios             2.943058
Sources             6.397953
Genres              0.000000
Duration            0.255918
Rating              0.000000
Score               0.000000
Scored by           0.000000
Members             0.000000
Favorites           0.000000
Description         0.767754
dtype: float64


We can easily drop 'Broadcast time' columns, because it refers to Japanese TV and MyAnimeList database consists mostly of data provided by foreign viewers. 

Term 'Starting season' only refers to TV series and cannot be applied to movies, OVAs and ONAs, so it is understandable that we have so many empty values.

Term 'End airing' can not be applied to currently airing for movies, OVAs and ONAs.

I assume that animes that don't have 'Producers' and 'Licensors' information are very unpopular and they can be dropped, but check is required.

**Decision:**
Divide TV, movies, OVAs and ONAs into different databases because they have useful attributes that can not be applied to other types of media.

In [8]:
#Dropping 'Broadcast time'
data = data.drop('Broadcast time', axis = 1)
data.head()

,Start air,End air,Title,Type,Episodes,Status,Starting season,Producers,Licensors,Studios,Sources,Genres,Duration,Rating,Score,Scored by,Members,Favorites,Description
0,2009-04-05,2010-07-04,Fullmetal Alchemist: Brotherhood,TV,64,Finished Airing,Spring,"Aniplex,Square Enix,Mainichi Broadcasting Syst...","Funimation,Aniplex of America",Bones,Manga,"Action,Military,Adventure,Comedy,Drama,Magic,F...",24 min. per ep.,R,9.25,719706,1176368,105387,"""In order for something to be obtained, someth..."
1,2016-08-26,NaT,Kimi no Na wa.,Movie,1,Finished Airing,NaN,"Kadokawa Shoten,Toho,Sound Team Don Juan,Lawso...","Funimation,NYAV Post",CoMix Wave Films,Original,"Supernatural,Drama,Romance,School",1 hr. 46 min.,PG-13,9.19,454969,705186,33936,"Mitsuha Miyamizu, a high school girl, yearns t..."
2,2015-04-08,2016-03-30,Gintama°,TV,51,Finished Airing,Spring,"TV Tokyo,Aniplex,Dentsu","Funimation,Crunchyroll",Bandai Namco Pictures,Manga,"Action,Comedy,Historical,Parody,Samurai,Sci-Fi...",24 min. per ep.,R,9.16,70279,194359,5597,"Gintoki, Shinpachi, and Kagura return as the f..."
3,2018-04-12,NaT,Steins;Gate 0,TV,23,Currently Airing,Spring,Nitroplus,Funimation,White Fox,Visual novel,"Sci-Fi,Thriller",23 min. per ep.,PG-13,9.16,12609,186331,1117,The dark untold story of Steins;Gate that lead...
4,2011-04-06,2011-09-14,Steins;Gate,TV,24,Finished Airing,Spring,"Frontier Works,Media Factory,Movic,AT-X,Kadoka...",Funimation,White Fox,Visual novel,"Sci-Fi,Thriller",24 min. per ep.,PG-13,9.14,552791,990419,90365,The self-proclaimed mad scientist Rintarou Oka...


In [9]:
#Checking popularity of animes without 'Producers' and 'Licensors'
data.loc[(data.Producers.isna())]

,Start air,End air,Title,Type,Episodes,Status,Starting season,Producers,Licensors,Studios,Sources,Genres,Duration,Rating,Score,Scored by,Members,Favorites,Description
76,1988-04-16,NaT,Hotaru no Haka,Movie,1,Finished Airing,NaN,NaN,"ADV Films,Central Park Media,Sentai Filmworks",Studio Ghibli,Novel,"Drama,Historical",1 hr. 28 min.,PG-13,8.55,144566,241170,2851,As World War II reaches its conclusion in 1945...
81,2015-08-20,2016-08-19,Kamisama Hajimemashita: Kako-hen,OVA,4,Finished Airing,NaN,NaN,NaN,TMS Entertainment,Manga,"Comedy,Demons,Supernatural,Romance,Fantasy,Shoujo",30 min. per ep.,PG-13,8.54,26535,60840,510,While playing in the snow one day at her shrin...
86,2012-08-26,NaT,Gintama: Yorinuki Gintama-san on Theater 2D,Movie,2,Finished Airing,NaN,NaN,NaN,Sunrise,Manga,"Action,Sci-Fi,Comedy,Historical,Parody,Samurai...",1 hr. 36 min. per ep.,PG-13,8.52,6891,18044,32,"Demonic Vice-Commander of the Shinsengumi, Tou..."
105,2017-09-27,2017-10-25,Natsume Yuujinchou Roku Specials,Special,2,Finished Airing,NaN,NaN,NaN,Shuka,Manga,"Slice of Life,Supernatural,Drama,Fantasy,Shoujo",23 min. per ep.,PG-13,8.48,4643,12547,21,New unaired episodes included in 4th and 5th D...
110,2016-12-09,NaT,Detective Conan: Episode One - The Great Detec...,Special,1,Finished Airing,NaN,NaN,TMS Entertainment,NaN,Manga,"Adventure,Mystery,Comedy,Police,Shounen",1 hr. 33 min.,PG-13,8.47,4174,9952,35,"The ""real"" episode 1 of Detective Conan writte..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1543,2014-06-07,NaT,Tamayura: More Aggressive - Tsuitachi dake no ...,Special,1,Finished Airing,NaN,NaN,NaN,TYO Animations,Original,"Comedy,Drama,Slice of Life",24 min.,G,7.49,1410,3424,1,Extra episode in the 7th Blu-ray and DVD volumes.
1548,2017-05-09,2017-09-22,Yao Shen Ji,ONA,40,Finished Airing,NaN,NaN,NaN,NaN,Novel,"Action,Adventure,Fantasy,Martial Arts,Romance,...",8 min. per ep.,PG-13,7.48,945,2395,27,"In his past life, although too weak to protect..."
1551,1990-08-25,NaT,City Hunter: Bay City Wars,Movie,1,Finished Airing,NaN,NaN,NaN,Sunrise,NaN,"Action,Comedy,Mystery,Shounen",45 min.,PG-13,7.48,1738,3673,1,A new island resort has been built in Tokyo ba...
1554,2015-10-07,NaT,Fate/stay night: Unlimited Blade Works 2nd Sea...,Special,1,Finished Airing,NaN,NaN,Aniplex of America,ufotable,Visual novel,"Action,Supernatural,Magic,Fantasy",9 min.,PG-13,7.48,25095,43716,122,Included with the second Blu-ray box set for t...


In [10]:
data.loc[(data.Licensors.isna())]

,Start air,End air,Title,Type,Episodes,Status,Starting season,Producers,Licensors,Studios,Sources,Genres,Duration,Rating,Score,Scored by,Members,Favorites,Description
6,2011-04-04,2012-03-26,Gintama',TV,51,Finished Airing,Spring,"TV Tokyo,Aniplex,Dentsu,Trinity Sound,Miracle ...",NaN,Sunrise,Manga,"Action,Sci-Fi,Comedy,Historical,Parody,Samurai...",24 min. per ep.,PG-13,9.11,90758,212238,4533,"After a one-year hiatus, Shinpachi Shimura ret..."
9,2012-10-04,2013-03-28,Gintama': Enchousen,TV,13,Finished Airing,Fall,"TV Tokyo,Aniplex,Dentsu,Shueisha,Miracle Bus",NaN,Sunrise,Manga,"Action,Comedy,Historical,Parody,Samurai,Sci-Fi...",24 min. per ep.,PG-13,9.07,62582,121612,1498,"While Gintoki Sakata was away, the Yorozuya fo..."
10,2013-07-06,NaT,Gintama Movie 2: Kanketsu-hen - Yorozuya yo Ei...,Movie,1,Finished Airing,NaN,"TV Tokyo,Aniplex,Dentsu,Bandai,Warner Bros.,Sh...",NaN,Sunrise,Manga,"Action,Sci-Fi,Comedy,Historical,Parody,Samurai...",1 hr. 50 min.,PG-13,9.04,60295,104021,1374,When Gintoki apprehends a movie pirate at a pr...
13,2017-01-09,2017-03-27,Gintama.,TV,12,Finished Airing,Winter,"TV Tokyo,Aniplex,Dentsu,Shueisha",NaN,Bandai Namco Pictures,Manga,"Action,Comedy,Historical,Parody,Samurai,Sci-Fi...",24 min. per ep.,PG-13,9.02,39493,89409,1222,After joining the resistance against the bakuf...
21,2017-01-06,NaT,Kizumonogatari III: Reiketsu-hen,Movie,1,Finished Airing,NaN,"Aniplex,Kodansha",NaN,Shaft,Light novel,"Action,Mystery,Supernatural,Vampire",1 hr. 23 min.,R,8.88,59924,125489,1477,After helping revive the legendary vampire Kis...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1551,1990-08-25,NaT,City Hunter: Bay City Wars,Movie,1,Finished Airing,NaN,NaN,NaN,Sunrise,NaN,"Action,Comedy,Mystery,Shounen",45 min.,PG-13,7.48,1738,3673,1,A new island resort has been built in Tokyo ba...
1552,2002-04-20,NaT,Crayon Shin-chan Movie 10: Arashi wo Yobu Appa...,Movie,1,Finished Airing,NaN,TV Asahi,NaN,NaN,Manga,"Historical,Drama,Romance,Samurai,Seinen",1 hr. 33 min.,G,7.48,978,2018,3,A full-scale historical drama set in the last ...
1557,1999-10-08,2000-03-31,Karakurizoushi Ayatsuri Sakon,TV,26,Finished Airing,Fall,WOWOW,NaN,TMS Entertainment,Manga,"Mystery,Horror,Thriller,Shounen",25 min. per ep.,R,7.48,2072,7405,39,"The dark-haired bishounen is Sakon Tachibana, ..."
1559,2013-08-06,NaT,Minami-ke Natsuyasumi,OVA,1,Finished Airing,NaN,Starchild Records,NaN,feel.,Manga,"Comedy,Slice of Life",28 min.,PG-13,7.48,6062,12111,4,Second Minami-ke OVA bundled with the 11th vol...


My assumption about popularity of those animes was wrong and some popular animes really don't have 'Producers' and 'Licensors'.

In [11]:
#3.  Dropping currently airing because they don't show final state of the score
data = data.loc[(data.Status == 'Finished Airing')]
data = data.drop('Status', axis = 1)
data.head()

,Start air,End air,Title,Type,Episodes,Starting season,Producers,Licensors,Studios,Sources,Genres,Duration,Rating,Score,Scored by,Members,Favorites,Description
0,2009-04-05,2010-07-04,Fullmetal Alchemist: Brotherhood,TV,64,Spring,"Aniplex,Square Enix,Mainichi Broadcasting Syst...","Funimation,Aniplex of America",Bones,Manga,"Action,Military,Adventure,Comedy,Drama,Magic,F...",24 min. per ep.,R,9.25,719706,1176368,105387,"""In order for something to be obtained, someth..."
1,2016-08-26,NaT,Kimi no Na wa.,Movie,1,NaN,"Kadokawa Shoten,Toho,Sound Team Don Juan,Lawso...","Funimation,NYAV Post",CoMix Wave Films,Original,"Supernatural,Drama,Romance,School",1 hr. 46 min.,PG-13,9.19,454969,705186,33936,"Mitsuha Miyamizu, a high school girl, yearns t..."
2,2015-04-08,2016-03-30,Gintama°,TV,51,Spring,"TV Tokyo,Aniplex,Dentsu","Funimation,Crunchyroll",Bandai Namco Pictures,Manga,"Action,Comedy,Historical,Parody,Samurai,Sci-Fi...",24 min. per ep.,R,9.16,70279,194359,5597,"Gintoki, Shinpachi, and Kagura return as the f..."
4,2011-04-06,2011-09-14,Steins;Gate,TV,24,Spring,"Frontier Works,Media Factory,Movic,AT-X,Kadoka...",Funimation,White Fox,Visual novel,"Sci-Fi,Thriller",24 min. per ep.,PG-13,9.14,552791,990419,90365,The self-proclaimed mad scientist Rintarou Oka...
5,1988-01-08,1997-03-17,Ginga Eiyuu Densetsu,OVA,110,NaN,Kitty Films,Sentai Filmworks,"Artland,Magic Bus",Novel,"Military,Sci-Fi,Space,Drama",26 min. per ep.,R,9.11,28452,121772,8370,The 150-year-long stalemate between the two in...


In [12]:
#4.  Changing dtype of 'Episodes' to int64
data = data.astype({'Episodes': int})
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1528 entries, 0 to 1562
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Start air        1514 non-null   datetime64[ns]
 1   End air          974 non-null    datetime64[ns]
 2   Title            1528 non-null   object        
 3   Type             1528 non-null   object        
 4   Episodes         1528 non-null   int32         
 5   Starting season  796 non-null    object        
 6   Producers        1165 non-null   object        
 7   Licensors        942 non-null    object        
 8   Studios          1483 non-null   object        
 9   Sources          1429 non-null   object        
 10  Genres           1528 non-null   object        
 11  Duration         1524 non-null   object        
 12  Rating           1528 non-null   object        
 13  Score            1528 non-null   float64       
 14  Scored by        1528 non-null   int64  

In [13]:
#5.  One-hot encode 'Genres', 'Producers', 'Licensors' and 'Studios'.

genres = data.set_index('Title').Genres.str.split(',', expand=True).stack()
data = pd.concat([data.set_index('Title').drop(['Genres'], axis = 1), pd.get_dummies(genres, prefix='g').groupby(level=0).sum()], axis = 1)

producers = data.Producers.str.split(',', expand=True).stack()
data = pd.concat([data.drop(['Producers'], axis = 1), pd.get_dummies(producers, prefix='p').groupby(level=0).sum()], axis = 1)

licensors = data.Licensors.str.split(',', expand=True).stack()
data = pd.concat([data.drop(['Licensors'], axis = 1), pd.get_dummies(licensors, prefix='p').groupby(level=0).sum()], axis = 1)

studios = data.Studios.str.split(',', expand=True).stack()
data = pd.concat([data.drop(['Studios'], axis = 1), pd.get_dummies(studios, prefix='p').groupby(level=0).sum()], axis = 1)

data.head()

,Start air,End air,Type,Episodes,Starting season,Sources,Duration,Rating,Score,Scored by,...,p_White Fox,p_Wit Studio,p_Xebec,p_Xebec Zwei,p_Yaoyorozu,p_Yumeta Company,p_Zexcs,p_feel.,p_production doA,p_ufotable
Fullmetal Alchemist: Brotherhood,2009-04-05,2010-07-04,TV,64,Spring,Manga,24 min. per ep.,R,9.25,719706,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Kimi no Na wa.,2016-08-26,NaT,Movie,1,NaN,Original,1 hr. 46 min.,PG-13,9.19,454969,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Gintama°,2015-04-08,2016-03-30,TV,51,Spring,Manga,24 min. per ep.,R,9.16,70279,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Steins;Gate,2011-04-06,2011-09-14,TV,24,Spring,Visual novel,24 min. per ep.,PG-13,9.14,552791,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Ginga Eiyuu Densetsu,1988-01-08,1997-03-17,OVA,110,NaN,Novel,26 min. per ep.,R,9.11,28452,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
#6.  Converting hours to minutes in 'duration' and change its dtype to int64

def convert_duration(string):
    pattern = re.findall(r'\d+', str(string))
    minutes_mul =  [a*int(b) for a,b in zip([1., 60.], list(reversed(pattern)))] #list is reversed becuase there isn't an hour value for each entry but minutes
    return sum(minutes_mul)

data['Duration'] = data['Duration'].apply(convert_duration)